In [ ]:
#############################
#                           #
#    Development Version    #
#                           #
#                           #
#       Nuno de Sousa       #
#       May 2020.           #
#############################

In [1]:
%matplotlib notebook

import numpy as np
#import ipyvolume as ipv
import scipy
import scipy.sparse.linalg as scslg
from datetime import datetime

import mole_geometries.Geom as Geom
import mole_mie.Materials as Materials

import numba as nb

import matplotlib.pyplot as plt

In [2]:
def random_cube(N_particles, edge_x = 1, edge_y = 1, edge_z = 1, spatial_translation = [0,0,0]):
    spatial_translation = np.array(spatial_translation)

    edges = np.array([edge_x, edge_y, edge_z]);

    positions = np.random.rand(N_particles,3)*edges - edges/2 + spatial_translation

    return positions

In [3]:
#G_tensor = np.eye(N_particles*3,dtype=complex)
#
#@jit(nopython=True)
#def SGreenTensor_numba(G_tensor, N_particles, alpha, pos, k):
#    for i in range(N_particles):
#        for j in range(i, N_particles):
#            if (i != j):
#
#                #Do lots of things, here is shown an example.
#                # However you should not be scared because 
#                #it only fills the G_tensor
#                #R = np.linalg.norm(np.array(pos[i])-np.array(pos[j]))
#                rx =pos[i][0]-pos[j][0]
#                ry =pos[i][1]-pos[j][1]
#                rz =pos[i][2]-pos[j][2]
#                R = np.sqrt(rx**2 + ry**2 + rz**2)
#                krq = (k*R)**2
#                pf = -k**2*alpha*np.exp(1j*k*R)/(4*np.pi*R)
#                a = 1.+(1j*k*R-1.)/(krq)
#                b = (3.-3.*1j*k*R-krq)/(krq) 
#                G_tensor[3*i,3*j] = pf*(a + b * (rx*rx)/(R**2))      #Gxx
#                G_tensor[3*i+1,3*j+1] = pf*(a + b * (ry*ry)/(R**2))  #Gyy
#                G_tensor[3*i+2,3*j+2] = pf*(a + b * (rz*rz)/(R**2))  #Gzz
#                G_tensor[3*i,3*j+1] = pf*(b * (rx*ry)/(R**2))        #Gxy
#                G_tensor[3*i,3*j+2] = pf*(b * (rx*rz)/(R**2))        #Gxz
#                G_tensor[3*i+1,3*j] = pf*(b * (ry*rx)/(R**2))        #Gyx
#                G_tensor[3*i+1,3*j+2] = pf*(b * (ry*rz)/(R**2))      #Gyz
#                G_tensor[3*i+2,3*j] = pf*(b * (rz*rx)/(R**2))        #Gzx
#                G_tensor[3*i+2,3*j+1] = pf*(b * (rz*ry)/(R**2))      #Gzy 
#                
#    return G_tensor

In [12]:
class CrossSections(object):

    """
    CrossSections calculation

    It contains the functions:
    - sigma_ext_calc - Calculation of the Extintion Cross Section
    - sigma_scatt_calc - Calculation of the Scattering Cross Section
    - sigma_abs_calc - Calculation of the Absorption Cross Section
    - relative_error_calc - It computes the relative error

    It returns the sections in the same units as the input.
    """
    
    sigma_scatt = None
    sigma_ext = None
    sigma_abs = None


    def sigma_ext_calc(self, epsilon_m, k, alpha, E0_const, epsilon_0 = 1, return_f = False):
        """
        It Computes the Extintion Cross Section.
        
        :param: epsilon_m = dielectric constant of the medium
        :param: k = wavenumber in free space
        :param: alpha = polarizability of the particles (it is a scalar)
        :param: E0_const = Magnitud of the electric field
        :param: epsilon_0 = dielectric permitivity of vacuum.
        :param: return_f = (True or False), if True returns a vector with the sigma_ext, otherwise it only writes in the class.
        """
        p = epsilon_0 * epsilon_m * alpha * self.E_inc

        self.sigma_ext = np.diagonal(
            k / (epsilon_0 * epsilon_m * E0_const ** 2) * np.imag(np.dot(np.conjugate(np.transpose(self.E_0i)), p)))
        if (return_f == True):
            return self.sigma_ext

    def sigma_scatt_calc(self, epsilon_m, k, alpha, E0_const, epsilon_0 = 1, return_f = False):
        """
        It Computes the Scattering Cross Section.
        
        :param: epsilon_m = dielectric constant of the medium
        :param: k = wavenumber in free space
        :param: alpha = polarizability of the particles (it is a scalar)
        :param: E0_const = Magnitud of the electric field
        :param: n_particles = Number of particles in the system.
        :param: epsilon_0 = dielectric permitivity of vacuum.
        :return: return_f = (True or False), if True returns a vector with the sigma_scatt, otherwise it only writes in the class.
        """
        
        n_particles = int(dda.SGreenTensor.shape[0]/3)
        p = epsilon_0 * epsilon_m * alpha * self.E_inc

        # diagonal parts
        diag = np.diag((k / (6 * np.pi)) * np.real(np.dot(np.transpose(np.conjugate(p)), p)))

        # nondiagonal parts
        i, j = np.mgrid[0:3 * n_particles, 0:3 * n_particles]
        tempG = self.SGreenTensor
        tempG[np.where(i == j)] = 0
        out_diag = np.dot(np.transpose(np.conjugate(p)),np.dot(np.imag(-(epsilon_m / (k ** 2 * alpha)) *tempG), p))
        self.sigma_scatt =  k ** 3 / (epsilon_0 * epsilon_m * E0_const) ** 2 * np.diagonal(np.real(out_diag + diag))

        if (return_f == True):
            return self.sigma_scatt

    def sigma_abs_calc(self, epsilon_m, k, alpha, E0_const, epsilon_0 = 1, return_f = False):
        """
        It Computes the Absorption Cross Section.

        :param: epsilon_m = dielectric constant of the medium
        :param: k = wavenumber in free space
        :param: alpha = polarizability of the particles (it is a scalar)
        :param: alpha0inv = Inverse of the static polarizability
        :param: E0_const = Magnitud of the electric field
        :return: return_f = (True or False), if True returns a vector with the sigma_ext, otherwise it only writes in the class.
        """
        
        alpha0inv = (1/alpha + 1j*k**3/(6*np.pi))
        
        p = epsilon_m * alpha * self.E_inc
        self.sigma_abs =  k / ((epsilon_m) ** 2 * E0_const ** 2) * np.diagonal(
            np.imag(np.dot(np.transpose(p), np.conjugate(alpha0inv * p))))

        if (return_f == True):
            return self.sigma_abs

    def relative_error_calc(self, return_f = False):
        """
        It checks the optical theorem
        """
        if((self.sigma_abs is not None) & (self.sigma_ext is not None) & (self.sigma_scatt is not None)):
            self.relative_error =  np.array((self.sigma_ext  - self.sigma_scatt- self.sigma_abs)/self.sigma_ext)

        else:
            raise ValueError("One or several sections are not calculated.")

        if (return_f == True):
            return self.relative_error

class Ilumination(object):

    """
    Ilumination
    ===========
    """
    
    E_0i = None  # Backgrounf Electromagnetic incident wave (usually a plane wave)
    
    def plane_wave(self, k, E0_const, positions, k_inc = 'z', return_f = True):
        """
        Plane Wave generator. This function generates the field experienced by a particle at some position in space.
        The wave must have a constant that characterizes the electric field (E0_const), the wave number and wave vector
        (k and k_inc).
        
        :param k: (float) Wavenumber
        :param positions: (np.array) positions of the particle
        :param E0_const:(float) Electric Field constant
        :param k_inc: direction of the wave vector
        :return: (np.array) Electric field in each particle with two different polarizations (sorted by x, y, z)
        """

        N_particles = positions.shape[0]

        if(k_inc == 'z'):
            kvector =k*np.array([0, 0, 1])
            Ex = np.zeros((N_particles,3),dtype=complex)
            Ey = np.zeros((N_particles,3),dtype=complex)
            i,x_i = np.mgrid[0:N_particles,0:3]
            j,y_j = np.mgrid[0:N_particles,0:3]
            Ex[np.where((x_i == 0))] = E0_const*np.exp(1j*np.array([np.dot(kvector,i) for i in positions]))
            Ey[np.where((y_j == 1))] = E0_const*np.exp(1j*np.array([np.dot(kvector,j) for j in positions]))
            Ex = Ex.reshape(3*N_particles)
            Ey = Ey.reshape(3*N_particles)
            E = np.transpose(np.vstack([Ex,Ey]))

        if(k_inc == 'y'):
            kvector =k*np.array([0, 1, 0])
            Ex = np.zeros((N_particles,3),dtype=complex)
            Ez = np.zeros((N_particles,3),dtype=complex)
            i,x_i = np.mgrid[0:N_particles,0:3]
            j,z_j = np.mgrid[0:N_particles,0:3]
            Ex[np.where((x_i == 0))] = E0_const*np.exp(1j*np.array([np.dot(kvector,i) for i in positions]))
            Ez[np.where((z_j == 2))] = E0_const*np.exp(1j*np.array([np.dot(kvector,j) for j in positions]))
            Ex = Ex.reshape(3*N_particles)
            Ez = Ez.reshape(3*N_particles)
            E = np.transpose(np.vstack([Ex,Ez]))

        if(k_inc == 'x'):
            kvector =k*np.array([1, 0, 0])
            Ey = np.zeros((N_particles,3),dtype=complex)
            Ez = np.zeros((N_particles,3),dtype=complex)
            i,y_i = np.mgrid[0:N_particles,0:3]
            j,z_j = np.mgrid[0:N_particles,0:3]
            Ex[np.where((y_i == 1))] = E0_const*np.exp(1j*np.array([np.dot(kvector,i) for i in positions]))
            Ey[np.where((z_j == 2))] = E0_const*np.exp(1j*np.array([np.dot(kvector,j) for j in positions]))
            Ey = Ey.reshape(3*N_particles)
            Ez = Ez.reshape(3*N_particles)
            E = np.transpose(np.vstack([Ey,Ez]))
            
        self.E_0i = E
        if(return_f == True):
            return self.E_0i
    
class Solver(object):
    
    def direct_solver(self, return_f = True):
        
        self.E_inc = scipy.linalg.solve(self.SGreenTensor, self.E_0i)
        if(return_f == True):
            return self.E_inc
    
    def bicgstab(self, return_f = True):
        scslg.bicgstab(self.SGreenTensor, self.E0_i)

    


class DDA(CrossSections, Ilumination, Solver):
    
    SGreenTensor = None
    
    def __init__(self):
        pass
    
    def SGreenTensorE_compute(self, positions, k, alpha, epsilon_m = 1, method = 'numpy', return_f = True):
        if(method == 'numpy'):
            np.seterr(divide='ignore', invalid='ignore')

            N_particles = positions.shape[0]

            G = np.zeros((N_particles, 3, N_particles, 3), dtype=complex)

            i, x_i, j, x_j = np.mgrid[0:N_particles, 0:3, 0:N_particles, 0:3]
            G[np.where((i == j) & (x_i == x_j))] = 1
            R = np.linalg.norm(positions[None, :, :] - positions[:, None, :], axis=-1)
            R = R.reshape(N_particles, 1, N_particles, 1)

            r = positions[None, :, :] - positions[:, None, :]

            krq = (k * R) ** 2
            pf = -k ** 2 / (epsilon_m) * alpha * np.exp(1j * k * R) / (4 * np.pi * R)
            a = 1. + (1j * k * R - 1.) / (krq)
            b = (3. - 3. * 1j * k * R - krq) / (krq)

            comb_r = (r[:, :, :, None] * r[:, :, None, :]).transpose([0, 2, 1, 3])
            G = pf * (b * comb_r / (R ** 2))
            G[np.where(x_i == x_j)] = (G + pf * a)[np.where(x_i == x_j)]
            G[np.where(i == j)] = 0
            G[np.where((i == j) & (x_i == x_j))] = 1

            self.SGreenTensor = G.reshape(N_particles * 3, N_particles * 3)

            if(return_f == True):
                return self.SGreenTensor
        
        elif(method == 'numba'):
            print("Not implemented.")
            #print('numba')
            #N_particles = positions.shape[0]
            #SGreenTensor = np.eye(N_particles*3,dtype=complex)
            
            #def SGreenTensor_numbax(self, SGreenTensor, N_particles, alpha, pos, k):
            #    self.SGreenTensor = self.SGreenTensor_numba(SGreenTensor, N_particles, alpha, pos, k)
            #print("abc")
            
            #self.SGreenTensor_numbax(SGreenTensor, N_particles, alpha, pos, k)
            
            #if(return_f == True):
            #    return self.SGreenTensor

    @staticmethod
    @nb.jit(nopython=True)
    def SGreenTensor_numba(G_tensor, N_particles, alpha, pos, k):
        for i in range(N_particles):
            for j in range(i, N_particles):
                if (i != j):

                    #Do lots of things, here is shown an example.
                    # However you should not be scared because 
                    #it only fills the G_tensor
                    #R = np.linalg.norm(np.array(pos[i])-np.array(pos[j]))
                    rx =pos[i][0]-pos[j][0]
                    ry =pos[i][1]-pos[j][1]
                    rz =pos[i][2]-pos[j][2]
                    R = np.sqrt(rx**2 + ry**2 + rz**2)
                    krq = (k*R)**2
                    pf = -k**2*alpha*np.exp(1j*k*R)/(4*np.pi*R)
                    a = 1.+(1j*k*R-1.)/(krq)
                    b = (3.-3.*1j*k*R-krq)/(krq) 
                    G_tensor[3*i,3*j] = pf*(a + b * (rx*rx)/(R**2))      #Gxx
                    G_tensor[3*i+1,3*j+1] = pf*(a + b * (ry*ry)/(R**2))  #Gyy
                    G_tensor[3*i+2,3*j+2] = pf*(a + b * (rz*rz)/(R**2))  #Gzz
                    G_tensor[3*i,3*j+1] = pf*(b * (rx*ry)/(R**2))        #Gxy
                    G_tensor[3*i,3*j+2] = pf*(b * (rx*rz)/(R**2))        #Gxz
                    G_tensor[3*i+1,3*j] = pf*(b * (ry*rx)/(R**2))        #Gyx
                    G_tensor[3*i+1,3*j+2] = pf*(b * (ry*rz)/(R**2))      #Gyz
                    G_tensor[3*i+2,3*j] = pf*(b * (rz*rx)/(R**2))        #Gzx
                    G_tensor[3*i+2,3*j+1] = pf*(b * (rz*ry)/(R**2))      #Gzy 

        return G_tensor

In [18]:
G

[array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 

In [13]:
#N_particles = 1000
#cube_edge = 10
#pos = random_cube(N_particles, edge_x=cube_edge, edge_y=cube_edge, edge_z=cube_edge)
#ipv.quickscatter(pos[:,0], pos[:,1], pos[:,2], size=2, marker="sphere")

In [14]:
pos = Geom.sphere_discretization(16, 230)
vol = pos[2]
pos = pos[0]
k = 2*np.pi/1260
alpha =  Materials.polarization(volume = vol, k = k, epsilon_part = 12.25, epsilon_medium = 1)[0]
N_particles = len(pos)
print("k = ", k)
print("alpha = ", alpha)
print("N_particles = ", N_particles)
#ipv.quickscatter(pos[:,0], pos[:,1], pos[:,2], size=2, marker="sphere")

k =  0.004986655005698084
alpha =  (56282.37304095568+20.83875203415576j)
N_particles =  2176


In [15]:
dda.SGreenTensor_numba(positions = pos, k = k, epsilon_m = 1, alpha = alpha, return_f = False)

TypeError: missing argument 'G_tensor'

In [8]:
dda = DDA()

%time dda.SGreenTensorE_compute(positions = pos, k = k, epsilon_m = 1, alpha = alpha, method = 'numpy', return_f = False)
%time dda.plane_wave(k = k, E0_const = 1 , positions = pos, k_inc = 'z', return_f = False)
%time dda.direct_solver(return_f = False)
#dda.bicgstab(return_f = False)

Wall time: 3.89 s
Wall time: 4.96 ms
Wall time: 3.71 s


In [ ]:
dda.sigma_ext_calc(epsilon_m = 1, k = k, alpha = alpha, E0_const = 1, return_f = False)
dda.sigma_scatt_calc(epsilon_m = 1, k = k, alpha = alpha, E0_const = 1, epsilon_0 = 1, return_f = False)
dda.sigma_abs_calc(epsilon_m = 1, k = k, alpha = alpha, E0_const = 1, epsilon_0 = 1, return_f = False)
print('sigma_ext = ', dda.sigma_ext)
print('sigma_scatt = ', dda.sigma_scatt)
print('sigma_abs = ', dda.sigma_abs)
print('relative error = ', dda.relative_error_calc(return_f=True))

In [ ]:
class Field(object):
    """
    CrossSections calculation

    It contains the functions:
    - Field - Calculation of the fields at r

    It returns the fields.
    """

    field_r = None

    def field_r_calc(self, k, alpha, E0_const, pos, r0, return_f=False):
        """
        It Computes scattered Field.

        :param: epsilon_m = dielectric constant of the medium
        :param: k = wavenumber in free space
        :param: alpha = polarizability of the particles (it is a scalar)
        :param: E0_const = Magnitud of the electric field
        :param: epsilon_0 = dielectric permitivity of vacuum.
        :param: pos = position of the particles
        :param: r0 = position where the field is observed
        :param: return_f = (True or False), if True returns a vector with the sigma_ext, otherwise it only writes in the class.
        """
        p = alpha * self.E_inc
	
        N_particles = pos.shape[0]
        N_r0 = r0.shape[0]
        
        G_tensor = np.empty((N_r0*3, N_particles*3), dtype = 'complex_')
        
        for i in range(N_particles):
            for j in range(N_r0):

		    # Do lots of things, here is shown an example.
		    # However you should not be scared because
		    # it only fills the G_tensor
		    # R = np.linalg.norm(np.array(pos[i])-np.array(pos[j]))
                rx = r0[j][0] - pos[i][0]
                ry = r0[j][1] - pos[i][1]
                rz = r0[j][2] - pos[i][2]
                R = np.sqrt(rx ** 2 + ry ** 2 + rz ** 2)
                krq = (k * R) ** 2
                pf = -k ** 2 * alpha * np.exp(1j * k * R) / (4 * np.pi * R)
                a = 1. + (1j * k * R - 1.) / (krq)
                b = (3. - 3. * 1j * k * R - krq) / (krq)
                G_tensor[3 * j + 0, 3 * i + 0] = pf * (a + b * (rx * rx) / (R ** 2))  # Gxx
                G_tensor[3 * j + 1, 3 * i + 1] = pf * (a + b * (ry * ry) / (R ** 2))  # Gyy
                G_tensor[3 * j + 2, 3 * i + 2] = pf * (a + b * (rz * rz) / (R ** 2))  # Gzz
                G_tensor[3 * j + 0, 3 * i + 1] = pf * (b * (rx * ry) / (R ** 2))  # Gxy
                G_tensor[3 * j + 0, 3 * i + 2] = pf * (b * (rx * rz) / (R ** 2))  # Gxz
                G_tensor[3 * j + 1, 3 * i + 0] = pf * (b * (ry * rx) / (R ** 2))  # Gyx
                G_tensor[3 * j + 1, 3 * i + 2] = pf * (b * (ry * rz) / (R ** 2))  # Gyz
                G_tensor[3 * j + 2, 3 * i + 0] = pf * (b * (rz * rx) / (R ** 2))  # Gzx
                G_tensor[3 * j + 2, 3 * i + 1] = pf * (b * (rz * ry) / (R ** 2))  # Gzy

        campo = np.dot(G_tensor, p)
        self.field_r = campo
        
        if (return_f == True):
            return self.field_r


                    
                    


In [ ]:
Field 

In [ ]:
%%time 
wl_list = np.arange(1000,2000,10)
pos = Geom.sphere_discretization(16, 230)
vol = pos[2]
pos = pos[0]
N_particles = len(pos)

results = []

for wl in wl_list:
    print("wavelength = ", wl)
    k = 2*np.pi/wl
    alpha =  Materials.polarization(volume = vol, k = k, epsilon_part = 12.25, epsilon_medium = 1)[0]
    
    dda = DDA()
    dda.SGreenTensorE_compute(positions = pos, k = k, epsilon_m = 1, alpha = alpha, return_f = False)
    dda.plane_wave(k = k, E0_const = 1 , positions = pos, k_inc = 'z', return_f = False)
    dda.direct_solver(return_f = False)
    
    dda.sigma_ext_calc(epsilon_m = 1, k = k, alpha = alpha, E0_const = 1, return_f = False)
    dda.sigma_scatt_calc(epsilon_m = 1, k = k, alpha = alpha, E0_const = 1, epsilon_0 = 1, return_f = False)
    dda.sigma_abs_calc(epsilon_m = 1, k = k, alpha = alpha, E0_const = 1, epsilon_0 = 1, return_f = False)
    dda.relative_error_calc(return_f = False)
    results.append([wl, dda.sigma_ext[0], dda.sigma_ext[1], dda.sigma_scatt[0], dda.sigma_scatt[1], dda.sigma_abs[0], dda.sigma_abs[1], dda.relative_error[0], dda.relative_error[1]])
    
results = np.array(results)

In [ ]:
fig = plt.figure(figsize = (9, 5.5))

sigma_geom = np.pi*230**2
plt.plot(results[:,0], results[:,1]/sigma_geom, label = '$\sigma_{ext}$', marker = '.', linewidth = 0.5)
plt.plot(results[:,0], results[:,3]/sigma_geom, label = '$\sigma_{ext}$', marker = 'x', linewidth = 0.5)
plt.plot(results[:,0], results[:,5]/sigma_geom, label = '$\sigma_{ext}$', marker = 'v', linewidth = 0.5)
plt.xlim(1000,2000)
plt.ylim(0,10)

--------

In [ ]:
%time
x = np.arange(-50000, 50000, 5000)
y = np.arange(-50000, 50000, 5000)
xx, yy = np.meshgrid(x, y)
z_pos = 10000
#Screen = np.array([list(xx.reshape(-1)),list(yy.reshape(-1)), z_pos+np.zeros(len(yy.reshape(-1)))]).T

In [ ]:
R = 10000
x = R*np.sin(np.linspace(0, 2*np.pi, 1000))
y = R*np.cos(np.linspace(0, 2*np.pi, 1000))
z = np.zeros(1000)
Screen = np.concatenate([[x,y,z]]).T

In [ ]:
dda.E_inc

In [ ]:
%%time 

def compute_electric_field(Screen, N_particles, pos, k):
    final_map = [] 
    for Screen_pos in Screen:
        temp_dist = Screen_pos - pos # distance vector
        R_vec = np.linalg.norm(temp_dist, axis = 1) # distances
        outer_R = (temp_dist[:,:,None]*temp_dist[:,None,:]) # outer product
        l_GT = np.einsum('i, ijk -> ijk', -((3-3*1j*k*R_vec - k**2*R_vec**2)/(k**2*R_vec**2)), outer_R) #left part of the Green Tensor
        r_GT = np.einsum('i, ijk -> ijk', 1 - (1j*k*R_vec-1)/(k**2*R_vec**2), np.array([np.eye(3) for x in range(N_particles)]))
        GT = np.einsum('i, ijk -> ijk', np.exp(1j*k*R_vec)/(4*np.pi*R_vec),l_GT + r_GT)
        E_temp = dda.E_inc[:,0].reshape(N_particles,3) # first polarization
        E_scatt = k**2*alpha*np.einsum('ijk, ij -> ik', GT, E_temp)
        E_scatt_total_vec = np.sum(E_scatt, axis = 0)
        E_scatt_total = E_scatt_total_vec.dot(np.conjugate(E_scatt_total_vec))
        final_map.append((Screen_pos[0], Screen_pos[1], Screen_pos[2], np.real(E_scatt_total)))

    final_map = np.array(final_map)
    
    return final_map

final_map = compute_electric_field(Screen, N_particles, pos, k)

In [ ]:
fig = plt.figure(figsize = (9, 5.5))

plt.polar(np.linspace(0, 2*np.pi, 1000), final_map[:,3]/np.max(final_map[:,3]))
plt.ylim(0,1.199)

In [ ]:
# Colormap
#fig = plt.figure(figsize = (9, 5.5))
#
#plt.pcolormesh(xx,yy,Z.reshape(20,20))

In [ ]:
import meshzoo
points, cells = meshzoo.octa_sphere(20)
ipv.quickscatter(points[:,0], points[:,1], points[:,2], size=2, marker="sphere")